In [27]:
import pandas as pd
import numpy as np

data = pd.read_csv("Bonds.csv")
data["Maturity Date"] = pd.to_datetime(data["Maturity Date"])
data["Trade Date"] = pd.to_datetime(data["Trade Date"])

data

,ISIN,Coupon,Issue Date,Maturity Date,Trade Date,Close Price
0,CA135087E679,1.50%,7/21/2015,2026-06-01,2026-01-05,99.680
1,CA135087E679,1.50%,7/21/2015,2026-06-01,2026-01-06,99.690
2,CA135087E679,1.50%,7/21/2015,2026-06-01,2026-01-07,99.690
3,CA135087E679,1.50%,7/21/2015,2026-06-01,2026-01-08,99.700
4,CA135087E679,1.50%,7/21/2015,2026-06-01,2026-01-09,99.700
...,...,...,...,...,...,...
415,CA135087XG49,5.75%,10/15/2001,2033-06-01,2026-01-12,116.340
416,CA135087XG49,5.75%,10/15/2001,2033-06-01,2026-01-13,116.290
417,CA135087XG49,5.75%,10/15/2001,2033-06-01,2026-01-14,116.533
418,CA135087XG49,5.75%,10/15/2001,2033-06-01,2026-01-15,116.950


In [28]:
selected_bonds = data[
    data["Maturity Date"].isin(
        pd.to_datetime(
            [
                "2026-09-01",
                "2027-03-01",
                "2027-09-01",
                "2028-03-01",
                "2028-09-01",
                "2029-03-01",
                "2029-09-01",
                "2030-03-01",
                "2030-09-01",
                "2031-03-01",
            ]
        )
    )
].copy()

selected_bonds["Maturity Period"] = (
    selected_bonds["Maturity Date"].rank(method="dense").astype(int)
)

selected_bonds["Coupon"] = (
    selected_bonds["Coupon"].str.rstrip("%").astype(float).astype(float)
) / 2
selected_bonds

,ISIN,Coupon,Issue Date,Maturity Date,Trade Date,Close Price,Maturity Period
60,CA135087L930,0.500,4/16/2021,2026-09-01,2026-01-05,99.150,1
61,CA135087L930,0.500,4/16/2021,2026-09-01,2026-01-06,99.150,1
62,CA135087L930,0.500,4/16/2021,2026-09-01,2026-01-07,99.170,1
63,CA135087L930,0.500,4/16/2021,2026-09-01,2026-01-08,99.160,1
64,CA135087L930,0.500,4/16/2021,2026-09-01,2026-01-09,99.190,1
...,...,...,...,...,...,...,...
345,CA135087T388,1.375,4/10/2025,2030-09-01,2026-01-12,99.210,9
346,CA135087T388,1.375,4/10/2025,2030-09-01,2026-01-13,99.185,9
347,CA135087T388,1.375,4/10/2025,2030-09-01,2026-01-14,99.355,9
348,CA135087T388,1.375,4/10/2025,2030-09-01,2026-01-15,99.365,9


In [29]:
import math
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

selected_bonds = pd.read_csv("Selected_Bonds.csv")


def build_lists(selected_bonds):
    """
    Build lists of bond in a specified format for processing.
    Return a dictionary of list of tuples, where each key is a trade date and
    each list is a list of bonds represented as tuples of
    (price, coupon_rate, maturity_period).
    """
    bond_dict = {}
    for trade_date in selected_bonds["Trade Date"].unique():
        bonds_on_date = selected_bonds[selected_bonds["Trade Date"] == trade_date]
        bond_list = []
        for _, row in bonds_on_date.iterrows():
            bond_tuple = (
                row["Close Price"],
                row["Coupon"],
                row["Maturity Period"],
            )
            bond_list.append(bond_tuple)
        bond_dict[trade_date] = bond_list
    return bond_dict


bond_dict = build_lists(selected_bonds)
bond_dict

{'2026-01-05': [(99.15, 0.5, 1),
  (98.6, 0.625, 2),
  (98.813, 1.375, 10),
  (100.22, 1.375, 3),
  (101.73, 1.75, 4),
  (101.34, 1.625, 5),
  (103.63, 2.0, 6),
  (102.22, 1.75, 7),
  (99.493, 1.375, 8),
  (99.165, 1.375, 9)],
 '2026-01-06': [(99.15, 0.5, 1),
  (98.63, 0.625, 2),
  (98.723, 1.375, 10),
  (100.3, 1.375, 3),
  (101.78, 1.75, 4),
  (101.41, 1.625, 5),
  (103.7, 2.0, 6),
  (102.33, 1.75, 7),
  (99.423, 1.375, 8),
  (99.085, 1.375, 9)],
 '2026-01-07': [(99.17, 0.5, 1),
  (98.66, 0.625, 2),
  (98.888, 1.375, 10),
  (100.28, 1.375, 3),
  (101.78, 1.75, 4),
  (101.4, 1.625, 5),
  (103.71, 2.0, 6),
  (102.37, 1.75, 7),
  (99.563, 1.375, 8),
  (99.245, 1.375, 9)],
 '2026-01-08': [(99.16, 0.5, 1),
  (98.67, 0.625, 2),
  (98.823, 1.375, 10),
  (100.31, 1.375, 3),
  (101.8, 1.75, 4),
  (101.43, 1.625, 5),
  (103.74, 2.0, 6),
  (102.34, 1.75, 7),
  (99.498, 1.375, 8),
  (99.17, 1.375, 9)],
 '2026-01-09': [(99.19, 0.5, 1),
  (98.67, 0.625, 2),
  (98.913, 1.375, 10),
  (100.3, 1.375, 

In [30]:
def calculate_ytm(bonds, face=100):
    """
    Calculate yield to maturity for a list of bonds.
    Each bond is represented as a tuple of (price, coupon_rate, maturity_period).
    Return a numpy array of YTMs corresponding to the input bonds.
    """

    def price_from_ytm(y, c, T):
        """Calculate the price of a bond given its yield to maturity (ytm), coupon rate, and maturity.
        It use the formula of present value of future cash flows.
        PV = sum_{t=1..T} c/(1+y)^t + face/(1+y)^T
        """
        if y <= -0.999999999999:
            return np.inf
        df = 1.0 / (1.0 + y)
        pv_coupons = c * (df * (1.0 - df**T) / (1.0 - df)) if c != 0 else 0.0
        pv_face = face * (df**T)
        return pv_coupons + pv_face

    ytms = np.zeros(len(bonds), dtype=float)

    for i, (price, coupon_rate, maturity) in enumerate(
        sorted(bonds, key=lambda x: x[2])
    ):
        P = float(price)
        c = float(coupon_rate)
        T = int(maturity)

        # Handle degenerate cases
        if T <= 0:
            ytms[i] = np.nan
            continue
        if P <= 0:
            ytms[i] = np.nan
            continue

        # Define root function
        def f(y):
            return price_from_ytm(y, c, T) - P

        # Bracket the root for bisection
        lo, hi = -0.95, 1.0  # start with -95% to 100%
        f_lo, f_hi = f(lo), f(hi)

        # Expand hi if needed
        if f_lo * f_hi > 0:
            for new_hi in [2.0, 5.0, 10.0]:
                f_hi = f(new_hi)
                if f_lo * f_hi <= 0:
                    hi = new_hi
                    break

        # If still not bracketed, return NaN (or you can raise)
        if f_lo * f_hi > 0:
            ytms[i] = np.nan
            continue

        # Bisection solve
        for _ in range(200):
            mid = 0.5 * (lo + hi)
            f_mid = f(mid)

            if abs(f_mid) < 1e-12:
                ytms[i] = mid
                break

            if f_lo * f_mid <= 0:
                hi = mid
                f_hi = f_mid
            else:
                lo = mid
                f_lo = f_mid
        else:
            ytms[i] = 0.5 * (lo + hi)

    return ytms


calculate_ytm(bond_dict["2026-01-05"])

array([0.01361573, 0.01339092, 0.01299752, 0.01303317, 0.01346081,
       0.01365753, 0.01414659, 0.01442558, 0.01474752, 0.01503737])

In [ ]:
ytm_dict = {}
for date_key, bonds in bond_dict.items():
    ytm_dict[date_key] = calculate_ytm(bonds)

ytm_dict

In [31]:
import numpy as np
import plotly.graph_objects as go

# If you already have ytms computed per date, make sure each ytm list
# corresponds to bonds sorted by maturity for that same date.

fig = go.Figure()

# Sort dates so lines appear in order
for date_key in sorted(bond_dict.keys()):
    bonds = bond_dict[date_key]

    # Sort bonds by maturity to get consistent x ordering
    bonds_sorted = sorted(bonds, key=lambda x: x[2])
    maturities = [b[2] / 2.0 for b in bonds_sorted]

    ytms = ytm_dict[date_key]

    fig.add_trace(
        go.Scatter(
            x=maturities,
            y=np.array(ytms, dtype=float) * 100.0,  # remove *100 if ytms already in %
            mode="lines+markers",
            name=date_key,
        )
    )

fig.update_layout(
    title="YTM Curves (Superimposed by Trade Date)",
    xaxis_title="Maturity (Years)",
    yaxis_title="YTM (%)",
)
fig.show()

In [32]:
def bootstrap_spot_rate_curve(bonds, face=100.0):
    """
    Bootstraps per-period spot rates from bonds.

    bonds: list of (price, coupon_cash_per_period, maturity_periods)
           maturity_periods = 1,2,...  (e.g., half-year periods if that's your setup)
           price should be DIRTY price in same units as face.
    Returns:
        spot_rates: array where spot_rates[k-1] is the spot rate for period k
                    (per-period rate, not annualized).
    """
    bonds_sorted = sorted(bonds, key=lambda x: x[2])
    n = len(bonds_sorted)
    spot_rates = np.zeros(n, dtype=float)

    for i, (price, coupon_per_period, maturity) in enumerate(bonds_sorted):
        P = float(price)
        c = float(coupon_per_period)
        T = int(maturity)

        # Cashflows at periods 1..T
        cash_flows = np.array([c] * (T - 1) + [face + c], dtype=float)

        # PV of known (earlier) cashflows using already-bootstrapped spot rates
        pv_known = 0.0
        for t in range(1, T):  # 1..T-1
            s_t = spot_rates[t - 1]  # spot for period t
            pv_known += cash_flows[t - 1] / ((1.0 + s_t) ** t)

        # Remaining PV allocated to the last cashflow at period T
        pv_last = P - pv_known

        if pv_last <= 0:
            print(
                f"Warning: Implied PV for last cashflow is non-positive at maturity {T}. "
                f"Using previous spot rate as fallback."
            )
            spot_rates[T - 1] = spot_rates[T - 2] if T - 2 >= 0 else np.nan
            continue

        # Solve discount factor at T: pv_last = CF_T / (1+s_T)^T
        cf_T = cash_flows[-1]
        spot_rate_T = (cf_T / pv_last) ** (1.0 / T) - 1.0

        spot_rates[T - 1] = spot_rate_T

    return spot_rates


bootstrap_spot_rate_curve(bond_dict["2026-01-05"])

array([0.01361573, 0.01339021, 0.01299106, 0.01302796, 0.01347101,
       0.0136779 , 0.01418811, 0.01446805, 0.0148038 , 0.01510695])

In [33]:
fig = go.Figure()  # reset every run

for date_key in sorted(bond_dict.keys()):
    bonds_sorted = sorted(bond_dict[date_key], key=lambda x: x[2])

    spot_rates = bootstrap_spot_rate_curve(bonds_sorted)
    maturities = [b[2] for b in bonds_sorted]  # or b[2]/2.0 for half-year periods

    fig.add_trace(
        go.Scatter(
            x=maturities,
            y=np.array(spot_rates, dtype=float) * 100.0,
            mode="lines+markers",
            name=str(date_key).strip(),
        )
    )

fig.update_layout(
    title="Bootstrapped Spot Rate Curves (Superimposed by Trade Date)",
    xaxis_title="Maturity (Years)",
    yaxis_title="Spot Rate (%)",
)
fig.show()

### Calculate Forward Rates

In [34]:
def forward_rates_1y_start_from_spot(
    spot_rates, periods_per_year=2, start_years=1.0, horizons_years=(1, 2, 3, 4)
):
    """
    spot_rates: array of per-period spot rates s_k for k=1..N (per-period, not annualized),
                where period length = 1/periods_per_year years.
    Returns: dict { "1y-1y": f, "1y-2y": f, ... } where f is per-period (same compounding as spot)
             forward rate over the horizon, starting at 1 year.
    """
    # Convert spot rates -> discount factors D_k
    # D_k = 1 / (1+s_k)^k
    D = np.array(
        [1.0 / ((1.0 + spot_rates[k - 1]) ** k) for k in range(1, len(spot_rates) + 1)],
        dtype=float,
    )

    a = int(
        round(start_years * periods_per_year)
    )  # start index in periods (e.g., 1y -> 2 periods)
    forwards = {}

    for h in horizons_years:
        b = int(round((start_years + h) * periods_per_year))  # end in periods
        if b > len(D):
            forwards[f"1y-{h}y"] = np.nan
            continue

        # Forward discount factor over (a,b): DF_fwd = D_b / D_a
        DF_fwd = D[b - 1] / D[a - 1]

        # Per-period forward rate over n = (b-a) periods: (1+f)^n = 1/DF_fwd
        n = b - a
        f = (DF_fwd ** (-1.0 / n)) - 1.0
        forwards[f"1y-{h}y"] = f

    return forwards

In [35]:
# ---- Plot forward curves superimposed by date ----
labels_order = ["1y-1y", "1y-2y", "1y-3y", "1y-4y"]
x_years = [1, 2, 3, 4]  # horizon length on x-axis (years)

fig = go.Figure()

for date_key in sorted(bond_dict.keys()):
    bonds_sorted = sorted(bond_dict[date_key], key=lambda x: x[2])

    # per-period spot rates from your fixed bootstrap
    spot_rates = bootstrap_spot_rate_curve(bonds_sorted)

    fwd = forward_rates_1y_start_from_spot(
        spot_rates, periods_per_year=2, start_years=1.0, horizons_years=(1, 2, 3, 4)
    )
    y = [fwd[k] * 100.0 for k in labels_order]  # percent

    fig.add_trace(
        go.Scatter(x=x_years, y=y, mode="lines+markers", name=str(date_key).strip())
    )

fig.update_layout(
    title="Forward Rates Starting in 1 Year (Superimposed by Trade Date)",
    xaxis_title="Forward Horizon (Years)  [1y-1y, 1y-2y, 1y-3y, 1y-4y]",
    yaxis_title="Forward Rate (%)",
)
fig.show()

#### YTM COV Matrix

In [36]:
def log_return_matrix(level_matrix):
    """
    level_matrix: shape (T, K) with strictly positive levels
    returns: shape (T-1, K) with log returns log(r_{t+1}/r_t)
    """
    X = np.asarray(level_matrix, dtype=float)
    if np.any(X <= 0):
        raise ValueError("All rates must be > 0 to take log-returns.")
    return np.log(X[1:, :] / X[:-1, :])


# ---- Build yield (YTM) matrix across dates ----
dates = sorted(bond_dict.keys())

# Use maturities 1..5 periods (or change this selection to match your Xi=1..5 definition)
# If your "maturity" is in half-year periods, 1..5 corresponds to 0.5y..2.5y.
maturity_targets = [1, 2, 3, 4, 5]

yield_levels = []
for d in dates:
    bonds_sorted = sorted(bond_dict[d], key=lambda x: x[2])

    ytms = np.array(calculate_ytm(bonds_sorted), dtype=float)  # decimals
    mats = np.array([b[2] for b in bonds_sorted], dtype=int)

    row = []
    for m in maturity_targets:
        idx = np.where(mats == m)[0]
        if len(idx) != 1:
            row.append(np.nan)
        else:
            row.append(ytms[idx[0]])
    yield_levels.append(row)

yield_levels = np.array(yield_levels, dtype=float)

# Drop dates with missing yields for these maturities
valid_rows = ~np.any(np.isnan(yield_levels), axis=1)
yield_levels = yield_levels[valid_rows, :]
yield_dates = [d for d, ok in zip(dates, valid_rows) if ok]

# Log-return time series Xi,j
yield_logrets = log_return_matrix(yield_levels)  # shape (T-1, 5)
cov_yields = np.cov(yield_logrets, rowvar=False, ddof=1)

cov_yields_df = pd.DataFrame(
    cov_yields,
    index=[f"X{i}" for i in maturity_targets],
    columns=[f"X{i}" for i in maturity_targets],
)

print("Covariance matrix of yield log-returns (Xi, i=1..5):")
print(cov_yields_df)

Covariance matrix of yield log-returns (Xi, i=1..5):
          X1        X2        X3        X4        X5
X1  0.000099  0.000003 -0.000047 -0.000024 -0.000028
X2  0.000003  0.000083  0.000026  0.000016  0.000024
X3 -0.000047  0.000026  0.000073  0.000035  0.000040
X4 -0.000024  0.000016  0.000035  0.000025  0.000024
X5 -0.000028  0.000024  0.000040  0.000024  0.000026


#### Forward COV Matrix

In [37]:
# ---- Build forward-rate matrix across dates ----
fwd_labels = ["1y-1y", "1y-2y", "1y-3y", "1y-4y"]

fwd_levels = []
fwd_dates = []
for d in dates:
    bonds_sorted = sorted(bond_dict[d], key=lambda x: x[2])
    spot = bootstrap_spot_rate_curve(bonds_sorted)

    fwd = forward_rates_1y_start_from_spot(
        spot_rates=spot,
        periods_per_year=2,
        start_years=1.0,
        horizons_years=(1, 2, 3, 4),
    )

    row = [fwd[k] for k in fwd_labels]
    if any((v is None) or np.isnan(v) for v in row):
        continue
    fwd_levels.append(row)
    fwd_dates.append(d)

fwd_levels = np.array(fwd_levels, dtype=float)

fwd_logrets = log_return_matrix(fwd_levels)  # shape (T-1, 4)
cov_forwards = np.cov(fwd_logrets, rowvar=False, ddof=1)

cov_forwards_df = pd.DataFrame(
    cov_forwards,
    index=fwd_labels,
    columns=fwd_labels,
)

print("\nCovariance matrix of forward-rate log-returns (1y-1y ... 1y-4y):")
print(cov_forwards_df)


Covariance matrix of forward-rate log-returns (1y-1y ... 1y-4y):
          1y-1y     1y-2y     1y-3y     1y-4y
1y-1y  0.000125  0.000071  0.000010 -0.000003
1y-2y  0.000071  0.000041  0.000005 -0.000003
1y-3y  0.000010  0.000005  0.000140  0.000121
1y-4y -0.000003 -0.000003  0.000121  0.000109


### Eigen Value and Vector

In [38]:
def eig_sorted(cov, labels):
    """
    Returns eigenvalues/eigenvectors sorted by descending eigenvalue.
    Eigenvectors are columns of V.
    """
    cov = np.asarray(cov, dtype=float)
    w, V = np.linalg.eigh(cov)  # for symmetric matrices (covariance)
    idx = np.argsort(w)[::-1]  # descending
    w = w[idx]
    V = V[:, idx]

    eigvals = pd.Series(w, index=[f"PC{i+1}" for i in range(len(w))], name="eigenvalue")
    eigvecs = pd.DataFrame(V, index=labels, columns=[f"PC{i+1}" for i in range(len(w))])
    return eigvals, eigvecs


def first_pc_one_sentence(eigvals, eigvecs, label_name="rates"):
    """
    Produces a single-sentence interpretation of the first principal component.
    """
    lam1 = float(eigvals.iloc[0])
    v1 = eigvecs.iloc[:, 0].values
    # Normalize sign so the largest-magnitude loading is positive (purely for readability)
    if v1[np.argmax(np.abs(v1))] < 0:
        v1 = -v1
    loadings = ", ".join([f"{lab}:{val:+.3f}" for lab, val in zip(eigvecs.index, v1)])
    return (
        f"The largest eigenvalue (PC1) is {lam1:.6g}, meaning the dominant source of variance in "
        f"daily log-returns of {label_name} is a single common factor whose loadings across maturities are "
        f"[{loadings}]."
    )


# ---- Use with your covariance matrices from Part 5 ----
# cov_yields_df, cov_forwards_df are from your previous step

# Labels for yields and forwards
yield_labels = list(cov_yields_df.index)  # e.g., ["X1","X2","X3","X4","X5"]
fwd_labels = list(cov_forwards_df.index)  # e.g., ["1y-1y","1y-2y","1y-3y","1y-4y"]

# Eigen-decompositions
eigvals_y, eigvecs_y = eig_sorted(cov_yields_df.values, yield_labels)
eigvals_f, eigvecs_f = eig_sorted(cov_forwards_df.values, fwd_labels)

print("=== YIELDS: eigenvalues ===")
print(eigvals_y)
print("\n=== YIELDS: eigenvectors (columns = PCs) ===")
print(eigvecs_y)

print("\n=== FORWARDS: eigenvalues ===")
print(eigvals_f)
print("\n=== FORWARDS: eigenvectors (columns = PCs) ===")
print(eigvecs_f)

# One-sentence interpretations for PC1
print("\nPC1 interpretation (yields):")
print(first_pc_one_sentence(eigvals_y, eigvecs_y, label_name="yields"))

print("\nPC1 interpretation (forwards):")
print(first_pc_one_sentence(eigvals_f, eigvecs_f, label_name="forward rates"))

=== YIELDS: eigenvalues ===
PC1    1.763767e-04
PC2    9.050976e-05
PC3    3.129880e-05
PC4    6.888546e-06
PC5    5.467699e-07
Name: eigenvalue, dtype: float64

=== YIELDS: eigenvectors (columns = PCs) ===
         PC1       PC2       PC3       PC4       PC5
X1 -0.580570 -0.564431  0.584707  0.007650  0.049139
X2  0.295577 -0.815084 -0.484129 -0.062753 -0.099766
X3  0.587393 -0.051574  0.556999 -0.551637 -0.194315
X4  0.316917 -0.059875  0.283917  0.786189 -0.444158
X5  0.360703 -0.103971  0.181328  0.271306  0.867524

=== FORWARDS: eigenvalues ===
PC1    0.000247
PC2    0.000166
PC3    0.000002
PC4    0.000001
Name: eigenvalue, dtype: float64

=== FORWARDS: eigenvectors (columns = PCs) ===
            PC1       PC2       PC3       PC4
1y-1y  0.062779 -0.864734  0.182740  0.463572
1y-2y  0.029657 -0.493226 -0.194715 -0.847310
1y-3y  0.750402  0.009313 -0.639273  0.167751
1y-4y  0.657325  0.094209  0.721126 -0.197550

PC1 interpretation (yields):
The largest eigenvalue (PC1) is 0.00017